In [1]:
from create_model import create_model
import torch
import torch.nn.utils.prune as prune
import numpy as np
import torch.nn as nn
from learn_that import apply_model

import torch.nn.functional as F

from copy import deepcopy as deepcopy

In [2]:
datasize = 2
inputSize = 3
middle = 1

In [3]:
np.random.seed(13)

In [4]:
X = 0.5-np.random.random((datasize, inputSize)) 

# TEST
X[1] = X[0]
# X = np.where(X == np.max(X), 1, 1)#np.zeros_like(X))
Y = 17 * np.random.random(datasize)

This is md

In [5]:
optim = "sparse_adam" # "adam"

In [6]:
torch.manual_seed(0)
model, optimizer, loss_fn = create_model(X, optim=optim, lr=1,task_type="regression", dropout=0.0,middle_layers=middle, first_layer=inputSize)

In [7]:
tx = torch.tensor(X)
tx = torch.max(tx, torch.zeros_like(tx))
ty = torch.tensor(Y)

In [8]:
tx


tensor([[0.0000, 0.2625, 0.0000],
        [0.0000, 0.2625, 0.0000]], dtype=torch.float64)

In [9]:
tx, ty = tx.type(torch.FloatTensor), ty.type(torch.FloatTensor)

In [10]:
parameters = list(model.named_parameters())
t = parameters[0]
arr = deepcopy(t[1].cpu().detach().numpy())
for p in parameters:
    print()
    print(p[1].shape)
    print(p[1])


torch.Size([3, 3])
Parameter containing:
tensor([[-0.0043,  0.3097, -0.4752],
        [-0.4249, -0.2224,  0.1548],
        [-0.0114,  0.4578, -0.0512]], requires_grad=True)

torch.Size([3])
Parameter containing:
tensor([ 0.1528, -0.1745, -0.1135], requires_grad=True)

torch.Size([1, 3])
Parameter containing:
tensor([[-0.5516, -0.3824, -0.2380]], requires_grad=True)

torch.Size([1])
Parameter containing:
tensor([0.0214], requires_grad=True)

torch.Size([3, 1])
Parameter containing:
tensor([[ 0.3953],
        [ 0.6000],
        [-0.6779]], requires_grad=True)

torch.Size([3])
Parameter containing:
tensor([-0.4355,  0.3632,  0.8304], requires_grad=True)

torch.Size([1, 3])
Parameter containing:
tensor([[-0.1188,  0.4320, -0.0931]], requires_grad=True)

torch.Size([1])
Parameter containing:
tensor([0.0611], requires_grad=True)


In [11]:
loss = loss_fn(apply_model(tx, model=model).squeeze(1), ty)

bwd = loss.backward()

In [12]:
for p in model.parameters():
    if optim == "sparse_adam":
        p.grad = p.grad.to_sparse() 

In [13]:
optimizer.step()
# optimizer.step(bwd)

In [14]:
parameters = list(model.named_parameters())
print("")
for p in parameters:
    print(p[0])
    print(p[1])


blocks.0.linear.weight
Parameter containing:
tensor([[-0.0043,  0.3097, -0.4752],
        [-0.4249, -0.2224,  0.1548],
        [-0.0114,  0.4578, -0.0512]], requires_grad=True)
blocks.0.linear.bias
Parameter containing:
tensor([ 0.1528, -0.1745, -0.1135], requires_grad=True)
blocks.1.linear.weight
Parameter containing:
tensor([[-0.5516, -0.3824, -0.2380]], requires_grad=True)
blocks.1.linear.bias
Parameter containing:
tensor([0.0214], requires_grad=True)
blocks.2.linear.weight
Parameter containing:
tensor([[ 0.3953],
        [ 0.6000],
        [-0.6779]], requires_grad=True)
blocks.2.linear.bias
Parameter containing:
tensor([-0.4355,  1.3632, -0.1696], requires_grad=True)
head.weight
Parameter containing:
tensor([[-0.1188,  1.4320,  0.9069]], requires_grad=True)
head.bias
Parameter containing:
tensor([1.0611], requires_grad=True)


In [15]:
parameters = list(model.named_parameters())
t = parameters[0]
arr2 = deepcopy(t[1].cpu().detach().numpy())


In [16]:
arr == arr2

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])